In [1]:
import math
from scipy.constants import Planck, alpha, e, epsilon_0, speed_of_light, m_p, m_n, c
from random import choices
import simpy as simp
import random as rm
import numpy as np

In [2]:
#function for caculating the electric dipole capture
def cap_E(l,R,v):
    M = 1
    h = (6.5821*(10**-16))/(10**6)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Qlmp = ((ke*h)/(M*c))*Qlm
    w = 2*math.pi*v
    E__amp = -((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Qlm+Qlmp))
    Ue = (c/(2*math.pi*(ke**2)))*(abs(E__amp)**2)
    return sig_cap_lm, Ue

#function for calculating the magnetic dipole caputure    
def cap_M(l,R,v):
    M = 1
    h = (6.5821*(10**-16))/(10**6)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Mlm = Qlm*(v/c)
    Mlmp = (math.sqrt((10*(h/(M*c*R))**2)*(abs(Qlm)**2)))+Mlm
    w = 2*math.pi*v
    M__amp = +((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Mlm+Mlmp))
    Um = (c/(2*math.pi*(ke**2)))*(abs(M__amp)**2)
    return sig_cap_lm, Um

#function for calculating the cross section of pair production
def pair(R,A,v,h):
    Z = A/2
    Na =  6.022*(10**23)
    X = 1433*(A/(Z*(Z+1)*(11.319-math.log(Z))))
    cross = (7/9)*(A/(X*Na))
    return cross

In [3]:
counts = 5000
Type = 'hw+D'
def neutron_collisions(env,counts,Type):
    M = 1
    E = 0
    Me = 0
    scat = 0
    testNum = 0
    l = 1
    A = 2
    R = (A**(1/3))*(10**-13)
    h = (6.5821*(10**-16))/(10**6)
    #for the formation of a duetron by neutron, proton capture
    if Type=='n+p':
        for i in range(counts+1):
            print('Test %d' % testNum)
            #neutron energy is randomly chosen from the range 8.5*10^10 and 10^11
            e = rm.randint(7*(10**18),1*(10**19))
            yield env.timeout(1)
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            react = ['cap E','cap M','scat']
            cross = [cap__E/(2*l +1),cap__M/(2*l + 1),1-((cap__E/(2*l +1))+(cap__M/(2*l + 1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e/9.223e+18,'Kilo Joule')
            print(win)
            print(cross)
            if win==['cap E']:
                E+=1
                print(UE,UE)
            if win==['scat']:
                scat+=1
            elif win==['cap M']:
                Me+=1
                print(UE,UE)
            print(E,Me,scat,'\n')
            testNum+=1
    #for photodisitegration of the deutron
    elif Type=='hw+D':
        E = 0
        Me = 0
        scat = 0
        ppro = 0
        for i in range(counts+1):
            #photon energy is randomly chosen from this range:
            print('Test',testNum)
            e = rm.randint(10,40)/(1*(10**19))
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            ke = ((2*math.pi*v)/c)
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            dis_E = cap__E*((k/ke)**2)
            dis_M = cap__M*((k/ke)**2)
            p_product = pair(R,A,v,h)
            react = ['dis E','dis M','com scattering','pair production']
            cross = [dis_E/(2*l +1),dis_M/(2*l + 1),abs(1-((dis_E/(2*l +1))+(dis_M/(2*l +1))+(p_product/(R**2*(math.pi))))),p_product/(R**2*(math.pi))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e*1e+6,'eV')
            print(win)
            print(cross)
            if win==['dis E']:
                E+=1
                print(UE,UM)
            elif win==['dis M']:
                Me+=1
                print(UE,UM)
            elif win==['com scattering']:
                scat+=1
            elif win==['pair production']:
                ppro += 1
            print(E,Me,scat,ppro,'\n')
            testNum+=1
env = simp.Environment()
env.process(neutron_collisions(env,counts,Type))
env.run(until=counts+2)

Test 0
2.4e-12 eV
['com scattering']
[0.2971637949276782, 2.251800499085585e-17, 0.2936979316302448, 0.4091382734420769]
0 0 1 0 

Test 1
3.6e-12 eV
['pair production']
[0.5459247507014038, 4.3904323170032267e-17, 0.044936975856519235, 0.4091382734420769]
0 0 1 1 

Test 2
3.7e-12 eV
['dis E']
[0.5688288558489311, 4.594495005006448e-17, 0.02203287070899207, 0.4091382734420769]
3.4768427067604965e-112 3.4768427067604965e-112
1 0 1 1 

Test 3
3.3e-12 eV
['dis E']
[0.47912622552359113, 3.8015871698777444e-17, 0.11173550103433194, 0.4091382734420769]
2.765728055268212e-112 2.765728055268212e-112
2 0 1 1 

Test 4
2.2e-12 eV
['pair production']
[0.26080328331521246, 1.9533231570409194e-17, 0.3300584432427106, 0.4091382734420769]
2 0 1 2 

Test 5
1.1e-12 eV
['dis E']
[0.09220788509395152, 6.371564499356043e-18, 0.4986538414639715, 0.4091382734420769]
3.073031172520233e-113 3.073031172520233e-113
3 0 1 2 

Test 6
1e-12 eV
['pair production']
[0.07992433533685375, 5.47072172098473e-18, 0.5109373

46 0 44 57 

Test 147
2.9e-12 eV
['pair production']
[0.39470809208243257, 3.071924844850869e-17, 0.1961536344754904, 0.4091382734420769]
46 0 44 58 

Test 148
2.9999999999999997e-12 eV
['dis E']
[0.41529902869380986, 3.24831100310742e-17, 0.17556269786411316, 0.4091382734420769]
2.2857256655109195e-112 2.2857256655109195e-112
47 0 44 58 

Test 149
1.3000000000000001e-12 eV
['pair production']
[0.1184660919240022, 8.330424281822376e-18, 0.47239563463392087, 0.4091382734420769]
47 0 44 59 

Test 150
2.5000000000000003e-12 eV
['dis E']
[0.3159286751744235, 2.4074599997054442e-17, 0.27493305138349955, 0.4091382734420769]
1.587309489938138e-112 1.587309489938138e-112
48 0 44 59 

Test 151
3.7e-12 eV
['dis E']
[0.5688288558489311, 4.594495005006448e-17, 0.02203287070899207, 0.4091382734420769]
3.4768427067604965e-112 3.4768427067604965e-112
49 0 44 59 

Test 152
3.3e-12 eV
['pair production']
[0.47912622552359113, 3.8015871698777444e-17, 0.11173550103433194, 0.4091382734420769]
49 0 44 60 


Test 328
1.6e-12 eV
['pair production']
[0.16175548168930473, 1.1643702050307509e-17, 0.42910624486861837, 0.4091382734420769]
107 0 97 125 

Test 329
3.7e-12 eV
['pair production']
[0.5688288558489311, 4.594495005006448e-17, 0.02203287070899207, 0.4091382734420769]
107 0 97 126 

Test 330
3.5e-12 eV
['dis E']
[0.5233365894682384, 4.190243466850127e-17, 0.06752513708968477, 0.4091382734420769]
3.1111266002787476e-112 3.1111266002787476e-112
108 0 97 126 

Test 331
4e-12 eV
['dis E']
[0.63939468269483, 5.2296938120182263e-17, 0.048532956136907, 0.4091382734420769]
4.063512294241633e-112 4.063512294241633e-112
109 0 97 126 

Test 332
3.8e-12 eV
['pair production']
[0.5920446053521343, 4.802407837626952e-17, 0.0011828787942111951, 0.4091382734420769]
109 0 97 127 

Test 333
3.1e-12 eV
['dis E']
[0.43623607187039976, 3.4287372141124085e-17, 0.15462565468752332, 0.4091382734420769]
2.440647071728882e-112 2.440647071728882e-112
110 0 97 127 

Test 334
3.7e-12 eV
['pair production']
[0.568828

3.862876374713454e-112 3.862876374713454e-112
163 0 138 203 

Test 504
1.4e-12 eV
['pair production']
[0.1323948484499297, 9.385911881325567e-18, 0.45846687810799336, 0.4091382734420769]
163 0 138 204 

Test 505
3.3e-12 eV
['dis E']
[0.47912622552359113, 3.8015871698777444e-17, 0.11173550103433194, 0.4091382734420769]
2.765728055268212e-112 2.765728055268212e-112
164 0 138 204 

Test 506
3.4e-12 eV
['pair production']
[0.5010688547432413, 3.993953156416941e-17, 0.08979287181468187, 0.4091382734420769]
164 0 138 205 

Test 507
2.6000000000000002e-12 eV
['dis E']
[0.3350727077605235, 2.5673416557680927e-17, 0.25578901879739957, 0.4091382734420769]
1.7168339443170898e-112 1.7168339443170898e-112
165 0 138 205 

Test 508
1e-12 eV
['com scattering']
[0.07992433533685375, 5.47072172098473e-18, 0.5109373912210693, 0.4091382734420769]
165 0 139 205 

Test 509
2.7e-12 eV
['pair production']
[0.3545885269965765, 2.7314048777603292e-17, 0.23627319956134651, 0.4091382734420769]
165 0 139 206 

Tes

207 0 184 264 

Test 655
3.1e-12 eV
['dis E']
[0.43623607187039976, 3.4287372141124085e-17, 0.15462565468752332, 0.4091382734420769]
2.440647071728882e-112 2.440647071728882e-112
208 0 184 264 

Test 656
1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.48579845929967813, 0.4091382734420769]
208 0 185 264 

Test 657
1.9e-12 eV
['pair production']
[0.2093193776047038, 1.5384482795173995e-17, 0.38154234895321926, 0.4091382734420769]
208 0 185 265 

Test 658
2.2e-12 eV
['com scattering']
[0.26080328331521246, 1.9533231570409194e-17, 0.3300584432427106, 0.4091382734420769]
208 0 186 265 

Test 659
2.6000000000000002e-12 eV
['dis E']
[0.3350727077605235, 2.5673416557680927e-17, 0.25578901879739957, 0.4091382734420769]
1.7168339443170898e-112 1.7168339443170898e-112
209 0 186 265 

Test 660
2.4e-12 eV
['com scattering']
[0.2971637949276782, 2.251800499085585e-17, 0.2936979316302448, 0.4091382734420769]
209 0 187 265 

Test 661
1.7e-12 eV
['com scattering']
[0.177154

...

1361 0 1136 1672 

Test 4169
2.2e-12 eV
['com scattering']
[0.26080328331521246, 1.9533231570409194e-17, 0.3300584432427106, 0.4091382734420769]
1361 0 1137 1672 

Test 4170
1.9e-12 eV
['pair production']
[0.2093193776047038, 1.5384482795173995e-17, 0.38154234895321926, 0.4091382734420769]
1361 0 1137 1673 

Test 4171
2.2e-12 eV
['pair production']
[0.26080328331521246, 1.9533231570409194e-17, 0.3300584432427106, 0.4091382734420769]
1361 0 1137 1674 

Test 4172
2.1e-12 eV
['pair production']
[0.2432248674565771, 1.81060018372879e-17, 0.3476368591013459, 0.4091382734420769]
1361 0 1137 1675 

Test 4173
2.9999999999999997e-12 eV
['pair production']
[0.41529902869380986, 3.24831100310742e-17, 0.17556269786411316, 0.4091382734420769]
1361 0 1137 1676 

Test 4174
3.8e-12 eV
['pair production']
[0.5920446053521343, 4.802407837626952e-17, 0.0011828787942111951, 0.4091382734420769]
1361 0 1137 1677 

Test 4175
1.8e-12 eV
['com scattering']
[0.19301354661926903, 1.4091364107556594e-17, 0.397848

Test 4346
2.3e-12 eV
['com scattering']
[0.2787858887116661, 2.1004062562752003e-17, 0.3120758378462569, 0.4091382734420769]
1421 0 1185 1741 

Test 4347
3.5e-12 eV
['pair production']
[0.5233365894682384, 4.190243466850127e-17, 0.06752513708968477, 0.4091382734420769]
1421 0 1185 1742 

Test 4348
1e-12 eV
['pair production']
[0.07992433533685375, 5.47072172098473e-18, 0.5109373912210693, 0.4091382734420769]
1421 0 1185 1743 

Test 4349
1.4999999999999999e-12 eV
['pair production']
[0.1468303797047897, 1.0490653819812366e-17, 0.4440313468531334, 0.4091382734420769]
1421 0 1185 1744 

Test 4350
3.4e-12 eV
['pair production']
[0.5010688547432413, 3.993953156416941e-17, 0.08979287181468187, 0.4091382734420769]
1421 0 1185 1745 

Test 4351
2.8e-12 eV
['pair production']
[0.3744691805324423, 2.899611353924436e-17, 0.21639254602548075, 0.4091382734420769]
1421 0 1185 1746 

Test 4352
1.8e-12 eV
['com scattering']
[0.19301354661926903, 1.4091364107556594e-17, 0.39784817993865396, 0.4091382734

2.9e-12 eV
['pair production']
[0.39470809208243257, 3.071924844850869e-17, 0.1961536344754904, 0.4091382734420769]
1474 0 1225 1803 

Test 4502
3.1e-12 eV
['pair production']
[0.43623607187039976, 3.4287372141124085e-17, 0.15462565468752332, 0.4091382734420769]
1474 0 1225 1804 

Test 4503
2.5000000000000003e-12 eV
['dis E']
[0.3159286751744235, 2.4074599997054442e-17, 0.27493305138349955, 0.4091382734420769]
1.587309489938138e-112 1.587309489938138e-112
1475 0 1225 1804 

Test 4504
2.8e-12 eV
['dis E']
[0.3744691805324423, 2.899611353924436e-17, 0.21639254602548075, 0.4091382734420769]
1.991121024178401e-112 1.991121024178401e-112
1476 0 1225 1804 

Test 4505
2.1e-12 eV
['pair production']
[0.2432248674565771, 1.81060018372879e-17, 0.3476368591013459, 0.4091382734420769]
1476 0 1225 1805 

Test 4506
2.1e-12 eV
['pair production']
[0.2432248674565771, 1.81060018372879e-17, 0.3476368591013459, 0.4091382734420769]
1476 0 1225 1806 

Test 4507
1.4999999999999999e-12 eV
['com scattering']

1511 0 1268 1874 

Test 4653
2.8e-12 eV
['dis E']
[0.3744691805324423, 2.899611353924436e-17, 0.21639254602548075, 0.4091382734420769]
1.991121024178401e-112 1.991121024178401e-112
1512 0 1268 1874 

Test 4654
2.9999999999999997e-12 eV
['pair production']
[0.41529902869380986, 3.24831100310742e-17, 0.17556269786411316, 0.4091382734420769]
1512 0 1268 1875 

Test 4655
3.8e-12 eV
['dis E']
[0.5920446053521343, 4.802407837626952e-17, 0.0011828787942111951, 0.4091382734420769]
3.6673198455530704e-112 3.6673198455530704e-112
1513 0 1268 1875 

Test 4656
1.7e-12 eV
['dis E']
[0.17715459251516152, 1.2844199016326209e-17, 0.4137071340427616, 0.4091382734420769]
7.339719081473946e-113 7.339719081473946e-113
1514 0 1268 1875 

Test 4657
2.6000000000000002e-12 eV
['dis E']
[0.3350727077605235, 2.5673416557680927e-17, 0.25578901879739957, 0.4091382734420769]
1.7168339443170898e-112 1.7168339443170898e-112
1515 0 1268 1875 

Test 4658
3.6e-12 eV
['pair production']
[0.5459247507014038, 4.3904323170

2e-12 eV
['com scattering']
[0.22606015799406753, 1.6722897801257884e-17, 0.3648015685638555, 0.4091382734420769]
1571 0 1317 1960 

Test 4848
1.4e-12 eV
['com scattering']
[0.1323948484499297, 9.385911881325567e-18, 0.45846687810799336, 0.4091382734420769]
1571 0 1318 1960 

Test 4849
1.9e-12 eV
['pair production']
[0.2093193776047038, 1.5384482795173995e-17, 0.38154234895321926, 0.4091382734420769]
1571 0 1318 1961 

Test 4850
2.2e-12 eV
['com scattering']
[0.26080328331521246, 1.9533231570409194e-17, 0.3300584432427106, 0.4091382734420769]
1571 0 1319 1961 

Test 4851
1.7e-12 eV
['dis E']
[0.17715459251516152, 1.2844199016326209e-17, 0.4137071340427616, 0.4091382734420769]
7.339719081473946e-113 7.339719081473946e-113
1572 0 1319 1961 

Test 4852
1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.48579845929967813, 0.4091382734420769]
1572 0 1320 1961 

Test 4853
1.1e-12 eV
['pair production']
[0.09220788509395152, 6.371564499356043e-18, 0.4986538414639715, 